In [1]:
import psycopg2
import pandas as pd
import numpy as np
import re
from datetime import datetime
import pytz
import re
from operator import itemgetter


VALID_FEATURES = [
    'pair_id','open_time','close_time','dow','tod',
    'open','high','low','close',
    'number_of_trades','volume','quote_asset_volume','taker_buy_base_asset_volume','taker_buy_quote_asset_volume',
    'ma14','ma30','ma90',
    'sup14','sup30','sup90',
    'res14','res30','res90',
    'atr','atr_diff','atr_ma14',
    'rsi','rsi_diff','rsi_ma14',
    'trend_up','trend_up3','trend_up14','trend_up30',
    'cs_ss','cs_ssr','cs_hm','cs_hmr','cs_brh','cs_buh','cs_ebu','cs_ebr'
]

In [2]:
# local postgres connection only
import hidden
sql_string = hidden.psycopg2(hidden.secrets())
print('PostgreSQL connection data taken from hidden.py')

# Make the connection and cursor
conn = psycopg2.connect(sql_string, connect_timeout=3)

PostgreSQL connection data taken from hidden.py


In [3]:
# These are only used for caching
import hashlib
import pickle

def get_batch_data(base_coin, quote_coin, start_time, end_time, columns, batch_size=30000, extra_rows=0, use_cache=True):
    column_info = [(x,) + re.match('^(?P<feature>[a-z][a-z0-9]*(?:_[a-z][a-z0-9]*)*)(?:_(?P<shift>[0-9]{1,3}))?$',x).groups() for x in columns]
    max_lookback = max([(0 if x==None else int(x)) for _,_,x in column_info])
    s = base_coin+quote_coin+f"{start_time}"+f"{end_time}"+"".join(columns)+str(batch_size)+str(extra_rows)
    h = hashlib.md5(s.encode('utf-8')).hexdigest()
    should_use_cache = use_cache and (datetime.strptime(end_time, '%Y-%m-%d') < datetime.now(pytz.timezone('UTC')).replace(tzinfo=None))
    if should_use_cache:
        # Can use cache
        try:
            with open(f'./cache_data/{h}.pkl', 'rb') as fp:
                print(f"Using cache file: ./cache_data/{h}.pkl")
                return pickle.load(fp)
        except:
            print(f"No cache found")
            pass

    sql = f"""
select
    f.*, open_time, open, high, low, close, volume, close_time, quote_asset_volume, number_of_trades, taker_buy_base_asset_volume, taker_buy_quote_asset_volume	
from
    (
        (select * from (select id as the_pair from pairs p where p.coin1='{base_coin}' and p.coin2='{quote_coin}') z inner join candlestick_15m on the_pair=pair_id where close_time notnull and open_time < '{start_time}' order by open_time desc limit {max_lookback + extra_rows})
            union all
        (select * from (select id as the_pair from pairs p where p.coin1='{base_coin}' and p.coin2='{quote_coin}') z inner join candlestick_15m on the_pair=pair_id where close_time notnull and open_time between '{start_time}' and '{end_time}' order by open_time limit {batch_size})
    ) cm
inner join 
    features f on f.pair_id = cm.pair_id and f.candle_open_time = cm.open_time
order by
    open_time desc
"""
    base_df = pd.read_sql_query(sql, conn)
    df = base_df[['candle_open_time']].copy()
    for name, feature, shift in column_info:
        assert feature in VALID_FEATURES, f"Invalid feature: {feature} for {name}"
        df[name] = base_df[feature].shift((0 if shift==None else -int(shift)))
        
    if extra_rows == 0:
        extra_df = None
    else:
        extra_df = df.copy()
        extra_df['is_extra'] = ~extra_df['candle_open_time'].between(start_time, end_time)
        extra_df = extra_df.set_index('candle_open_time').sort_index()
        
    df = df[df['candle_open_time'].between(start_time, end_time)]
    df = df.set_index('candle_open_time').sort_index()

    ref_df = base_df[['open_time','open', 'high', 'low', 'close']].copy()
    ref_df = ref_df[ref_df['open_time'].between(start_time, end_time)]
    ref_df = ref_df.set_index('open_time').sort_index()

    batch_close_time = base_df['close_time'].max()
    
    if should_use_cache:
        with open(f'./cache_data/{h}.pkl', 'wb') as fp:
            print(f"Saving cache to: ./cache_data/{h}.pkl")
            pickle.dump((df, ref_df, extra_df, batch_close_time), fp, protocol=4)

    return df, ref_df, extra_df, batch_close_time

In [4]:
repeat_columns = ['high', 'low', 'close','rsi',
    'trend_up','trend_up3','trend_up14','trend_up30',
    'cs_ss','cs_ssr','cs_hm','cs_hmr','cs_brh','cs_buh','cs_ebu','cs_ebr']
static_columns = ['open']

columns = static_columns + [f"{rc}_{i}" for rc in repeat_columns for i in range(0,24)]

In [16]:
repeat_columns

['high',
 'low',
 'close',
 'rsi',
 'trend_up',
 'trend_up3',
 'trend_up14',
 'trend_up30',
 'cs_ss',
 'cs_ssr',
 'cs_hm',
 'cs_hmr',
 'cs_brh',
 'cs_buh',
 'cs_ebu',
 'cs_ebr']

In [17]:
len(repeat_columns)

16

In [6]:
mapping = {
    'ETHBTC':0,
    'BTCUSDT':1,
    'ETHUSDT':2,
    'BTCETH':-1,
    'USDTBTC':-2,
    'USDTETH':-3
}

In [7]:
from itertools import permutations

for a,b in permutations(['ETH','BTC','USDT'],2):
    df, ref_df, _, _ = get_batch_data(a, b, '2018-01-01', '2021-08-07', columns, 500000, 0, True)
    df[["r_"+x for x in df.columns if x.startswith('rsi_')]] = df[[x for x in df.columns if x.startswith('rsi_')]] < 30
    df[[x for x in df.columns if x.startswith('rsi_')]] = df[[x for x in df.columns if x.startswith('rsi_')]] > 70

#     for col in df.columns:
#         if df[col].dtype.kind in ['O']:
#             df[col] = df[col].astype(float)
    df = df.astype(float)
            
    with open(f'./cache_data/{a}_{b}_f4_406.pkl', 'wb') as fp:
        pickle.dump(df, fp, protocol=4)

No cache found
Saving cache to: ./cache_data/31540be8258b739d4961aebd9d62ea1c.pkl
No cache found
Saving cache to: ./cache_data/cd3862dfd26f3666dc1bf3b6c90e6aee.pkl
No cache found
Saving cache to: ./cache_data/0246979fa9db8a28a785d5c6bcf9dfce.pkl
No cache found
Saving cache to: ./cache_data/85478b2a84ed13059d45744fd2805e3e.pkl
No cache found
Saving cache to: ./cache_data/441f88e6601c92c89c79a631532c2c51.pkl
No cache found
Saving cache to: ./cache_data/8c723b2ec136f6dcfbf3a79ddc3ebf96.pkl


In [8]:
with open(f'./cache_data/targets.pkl', 'rb') as fp:
    y_df = pickle.load(fp)
    y_df = y_df.reset_index()

In [9]:
y_df

,pair_id,open_time,close__ewm_8,close__ewm_24,close__ewm_48,close__ewm_96
0,0,2017-07-14 04:00:00,-0.085983,0.314472,0.223167,0.589414
1,0,2017-07-14 04:15:00,0.094960,0.472451,0.347574,0.742649
2,0,2017-07-14 04:30:00,0.024124,0.372090,0.221948,0.632886
3,0,2017-07-14 04:45:00,0.056716,0.376667,0.209570,0.636785
4,0,2017-07-14 05:00:00,0.503308,0.793327,0.602286,1.053142
...,...,...,...,...,...,...
839094,1,2021-08-06 22:45:00,NaN,NaN,NaN,NaN
839095,1,2021-08-06 23:00:00,NaN,NaN,NaN,NaN
839096,1,2021-08-06 23:15:00,NaN,NaN,NaN,NaN
839097,1,2021-08-06 23:30:00,NaN,NaN,NaN,NaN


In [10]:
dfs = []
for a,b in permutations(['ETH','BTC','USDT'],2):
    with open(f'./cache_data/{a}_{b}_f4_406.pkl', 'rb') as fp:
        df = pickle.load(fp)
        df['pair_id'] = mapping[f"{a}{b}"]
        df = df.reset_index().set_index(['pair_id','candle_open_time'])
        df = df.merge(y_df, left_on=['pair_id','candle_open_time'], right_on=['pair_id','open_time'])
        dfs.append(df)
merged_df = pd.concat(dfs)
merged_df = merged_df.dropna()
merged_df = merged_df.set_index(['pair_id','open_time'])
dfs = []

In [11]:
with open(f'../data/tc0_full.pkl', 'wb') as fp:
    pickle.dump(merged_df, fp, protocol=4)

In [12]:
with open(f'../data/tc0_train.pkl', 'wb') as fp:
    pickle.dump((merged_df[merged_df.index.get_level_values(1) < '2021-01-01']).reset_index(drop=True), fp, protocol=4)

In [13]:
with open(f'../data/tc0_test.pkl', 'wb') as fp:
    pickle.dump((merged_df[merged_df.index.get_level_values(1) >= '2021-01-01']).reset_index(drop=True), fp, protocol=4)

In [23]:
[x for x in merged_df.columns if x not in columns]

['r_rsi_0',
 'r_rsi_1',
 'r_rsi_2',
 'r_rsi_3',
 'r_rsi_4',
 'r_rsi_5',
 'r_rsi_6',
 'r_rsi_7',
 'r_rsi_8',
 'r_rsi_9',
 'r_rsi_10',
 'r_rsi_11',
 'r_rsi_12',
 'r_rsi_13',
 'r_rsi_14',
 'r_rsi_15',
 'r_rsi_16',
 'r_rsi_17',
 'r_rsi_18',
 'r_rsi_19',
 'r_rsi_20',
 'r_rsi_21',
 'r_rsi_22',
 'r_rsi_23',
 'close__ewm_8',
 'close__ewm_24',
 'close__ewm_48',
 'close__ewm_96']